# Re-reconsidering the negative income tax

Revisits [Wiederspan, Rhodes and Shaefer (2015)](https://drive.google.com/file/d/0B5yUTVqCinNRdWo3YVlhZFZIRlU/view?usp=sharing): *Expanding the Discourse on Antipoverty Policy: Reconsidering a Negative Income Tax*.

This paper with a negative income tax these programs, using :
* EITC
* SSI
* SNAP
* TANF
* School meal programs
* Housing subsidies

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-02-20*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# On Python 3.6 use "import urllib.request as url_lib".
import urllib as url_lib

In [2]:
tc.__version__

'0.17.0'

### Settings

In [3]:
sns.set_style('white')
DPI = 75
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi'] = DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [5]:
# Show one decimal in tables.
pd.set_option('precision', 2)

### Utilities

From https://stackoverflow.com/a/48999797/1840471.

In [6]:
def gini(x, w=None):
    # Array indexing requires reset indexes.
    x = pd.Series(x).reset_index(drop=True)
    if w is None:
        w = np.ones_like(x)
    w = pd.Series(w).reset_index(drop=True)
    n = x.size
    wxsum = sum(w * x)
    wsum = sum(w)
    sxw = np.argsort(x)
    sx = x[sxw] * w[sxw]
    sw = w[sxw]
    pxi = np.cumsum(sx) / wxsum
    pci = np.cumsum(sw) / wsum
    g = 0.0
    for i in np.arange(1, n):
        g = g + pxi.iloc[i] * pci.iloc[i - 1] - pci.iloc[i] * pxi.iloc[i - 1]
    return g

2018 HHS Poverty Guidelines (Federal Poverty Level) for 48 Contiguous States and District of Columbia (Alaska and Hawaii have higher guidelines, but the data does not have state code): https://aspe.hhs.gov/poverty-guidelines

In [7]:
def fpl(XTOT):
    return 7820 + 4320 * XTOT

In [8]:
def weight(df, col):
    return df[col] * df.s006

def weighted_sum(df, col):
    return (weight(df, col)).sum()

def weighted_mean(df, col):
    return weighted_sum(df, col) / df.s006.sum()

## Data

In [9]:
recs = tc.Records.cps_constructor()
calc = tc.Calculator(records=recs, policy=tc.Policy(), verbose=False)
calc.advance_to_year(2018)
calc.calc_all()

TODO: Try a reform with EITC repeal in case it affects other credits.

In [10]:
BENS = ['eitc', 'ssi_ben', 'housing_ben', 'snap_ben', 'tanf_ben']

In [11]:
df = calc.dataframe(['s006', 'XTOT', 'expanded_income', 'aftertax_income'] +
                    BENS)

In [12]:
for i in BENS:
    df[i + '_b'] = weight(df, i) / 1e9
    print(i, round(df[i + '_b'].sum(), 1))

('eitc', 59.6)
('ssi_ben', 57.0)
('housing_ben', 2.4)
('snap_ben', 78.9)
('tanf_ben', 7.8)


## Exploratory

Current distribution of benefits programs as listed in Wiederspan et al:

![fd](https://i.imgur.com/LrYROJp.png)

These are:
* `eitc`
* `ssi_ben`
* `housing_ben` (too low: https://github.com/open-source-economics/C-TAM/issues/66)
* `snap_ben`
* `tanf_ben` (too low: https://github.com/open-source-economics/C-TAM/issues/65)

School food programs are not explicitly modeled.